In [2]:
#Installing the Python Package
!pip install SPARQLWrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.8 MB/s eta 0:00:00


In [4]:
#Importing the Python Package
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

#Defining the SPARQL Endpoint of Wikidata
endpoint_url = "https://query.wikidata.org/sparql"

#Defining the SPARQL Query
query = """SELECT * WITH {
  SELECT * WHERE {
  ?item wdt:P31 wd:Q32815.
  ?item wdt:P17 ?country.
  ?item wdt:P625 ?coord.
  }
 }
AS %list
WHERE {
  INCLUDE %list.
  OPTIONAL{?item rdfs:label ?label.
  FILTER(LANG(?label)="en")}
  OPTIONAL{?item wdt:P3896 ?geoshape.}
  OPTIONAL{?item wdt:P51 ?audio. }
  OPTIONAL{?item wdt:P18 ?image. }
  OPTIONAL{?item wdt:P10 ?video. }
  OPTIONAL{?item wikibase:sitelinks ?url.}
  }"""

#Defining a function to retrieve query results
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

#Creating the header of the GeoJSON
with open("map.geojson", "w") as f:
  f.write("""{
"type": "FeatureCollection",

"features": [""")

#Retrieving the data from the results
i = 0
qids = []
results = get_results(endpoint_url, query)
with open("map.geojson", "a") as f:
  for result in results["results"]["bindings"]:
    qid = result["item"]["value"][31:]
    if (qid in qids): continue
    qids.append(qid)
    coord = result["coord"]["value"].replace("Point(","").replace(")", "").replace(" ", ", ")
    if (coord.count(" ") == 0): continue
    country = result["country"]["value"][31:]
    #coor = coord.split(" ")
    #coord = coor[1] + ", " + coor[0]
    nsitelinks = int(result["url"]["value"]) != 0
    try:
      name = result["label"]["value"]
    except KeyError:
      name = "*"+qid
    try:
      audio = result["audio"]["value"]
    except KeyError:
      audio = ""
    try:
      video = result["video"]["value"]
    except KeyError:
      video = ""
    try:
      image = result["image"]["value"]
    except KeyError:
      image = ""
    try:
      geoshape = result["geoshape"]["value"]
    except KeyError:
      geoshape = ""
    print(qid, coord, country, audio, video, image, geoshape)
    #Converting the data into GeoJSON layout
    f.write('''{ "type": "Feature", "id": '''+ str(i) +''', "properties": { "NAME": "'''+ name +'''", "COUNTRY": "'''+ country +'''", "AUDIO": "'''+ audio +'''", "VIDEO": "'''+ video +'''", "IMAGE": "'''+ image +'''", "GEOSHAPE": "'''+ geoshape +'''", "QID": "'''+qid+'''", "WIKIPEDIA": "'''+str(nsitelinks)+'''" }, "geometry": { "type": "Point", "coordinates": [ '''+ coord +''' ] } }''')
    if results["results"]["bindings"].index(result) < len(results["results"]["bindings"]) - 1:
      f.write(",\n")
    i += 1
  #Adding the ending part of GeoJSON
  f.write("""\n]
}""")

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Q4022506 13.59791389, 46.40491111 Q215   http://commons.wikimedia.org/wiki/Special:FilePath/Mo%C5%A1eja-Log%20pod%20Mangartom4.jpg 
Q4022674 121.25376389, 24.92976667 Q865   http://commons.wikimedia.org/wiki/Special:FilePath/Longgang%20Mosque.JPG 
Q4022677 32.633333333, 0.3 Q1036   http://commons.wikimedia.org/wiki/Special:FilePath/Aerial%20view%20of%20the%20Gaddafi%20mosque%20in%20Uganda.jpg 
Q4022684 20.737899, 42.208297 Q1246   http://commons.wikimedia.org/wiki/Special:FilePath/PrizrenCollection2%202010%20100%202517.JPG 
Q40376648 44.581111111, 33.098333333 Q796   http://commons.wikimedia.org/wiki/Special:FilePath/Salman%20the%20Persian%20tomb%20in%20Ctesiphon%20-%20Madain%20-%20Iraq.png 
Q40377967 31.2944055, 30.083415 Q79   http://commons.wikimedia.org/wiki/Special:FilePath/Gamal%20Abdel%20Nasser%20Mosque1.jpg 
Q41010756 21.786166666, 41.7105 Q221   http://commons.wikimedia.org/wiki/Special:FilePath/01%2